In [186]:
import requests
import datetime
import pandas as pd
import numpy as np

# Stock Price Prediction Using GANs

## Overview
The goal is to take a number of past and conscutive stock prices and predict the price of the next day. For example, if we take a time series from 1 to 't', we will predict the t+1 price.
One of the greatest strengths of a GAN is the fact that it can extract features in a sophisticated manner and replicate the distribution of the real data.

The proposed idea is to use Generative Adversarial Network (GAN) with Convolutional Neural Network (CNN) as the discriminator and Long Short-Term memory (LSTM) as the genertor for predicting the stock price on t+1 day.

## Data

The idea is to start the project with a simple set of data and keep adding features and data sources to help the model extract useful information of the market.

As the first step, we will start with just getting the stock price information from `finnhub.io`
We will split the data to train/test sets in a manner that is acceptable to for time series data.

## Strategy

Our strategy will be to split the data into batches of 16 stock prices and then feed the first 15 of them to the Generator and have the LSTM model to predict the 16th day. But, before feeding the data, we ill normalize each batch.

Another strategy is to feed a Gaussian distribution to the Generator and have it learn the 16 prices.
The first strategy should be a better fit for us because we want to control the distribution of the first 15 days and not feed randomly.

In [187]:
def from_date_to_timestamp(date_string: str) -> str:
    date = datetime.datetime.strptime(date_string, "%m/%d/%Y")
    return datetime.datetime.timestamp(date)

In [188]:
to_time = from_date_to_timestamp("8/19/2020")
from_time =from_date_to_timestamp("8/19/2019")

In [189]:
response = requests.get(
    'https://finnhub.io/api/v1/stock/candle?symbol=AAPL&resolution=1&from={}&to={}'.format(from_time, to_time),
#     params={'q': 'requests+language:python'},
    headers={'X-Finnhub-Token': 'bt0udmn48v6qcviaikf0'}
)

# response = requests.get(/stock/candle?symbol=AAPL&resolution=1&from=1572651390&to=1572910590&token=bt0udmn48v6qcviaikf0')

# View the new `text-matches` array which provides information
# about your search term within the results
json_response = response.json()
data = pd.DataFrame(data=json_response)
data.columns = ["Close", "High", "Low", "Open", "Status", "DateTime", "Volume"]
data.drop(['Status'], axis=1, inplace=True) #not needed column
# repository = json_response['items'][0]
# print(f'Text matches: {repository["text_matches"]}')

In [190]:
# convert timestamp to datetime
data['DateTime'] = data['DateTime'].apply(lambda t: datetime.datetime.fromtimestamp(t))

In [191]:
#TODO partition the data in year and month and save

In [192]:
data.head()

,Close,High,Low,Open,DateTime,Volume
0,317.460,317.4699,317.08,317.120,2020-05-12 17:32:00,85768
1,317.490,317.5700,317.39,317.420,2020-05-12 17:33:00,92240
2,317.560,317.6098,317.42,317.480,2020-05-12 17:34:00,54167
3,317.480,317.5900,317.45,317.555,2020-05-12 17:35:00,45673
4,317.446,317.5401,317.40,317.500,2020-05-12 17:36:00,44307


In [193]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import numpy as np

def batch_data(prices, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    n_batches = len(prices)//batch_size
    
    # only full batches
    prices = prices[:n_batches*batch_size]
    
    features, target = [], []
    
    for idx in range(0, (len(prices) - sequence_length)):
        features.append(prices[idx : idx + sequence_length])
        target.append(prices[idx + sequence_length])
    
    data = TensorDataset(torch.from_numpy(np.asarray(features)), torch.from_numpy(np.asarray(target)))
    data_loader = DataLoader(data, shuffle= False, batch_size = batch_size)
    # return a dataloader
    return data_loader

In [194]:
# test dataloader

test_prices = range(50)
t_loader = batch_data(test_prices, sequence_length=16, batch_size=50)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([34, 16])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
        [ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
        [ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
        [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
        [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
        [ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
        [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
        [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
        [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
        [13, 14, 

In [161]:
def normalizeBatch(data_loader):
    min_val = data_loader.min(1, keepdim=True)[0]
    max_val = data_loader.max(1, keepdim=True)[0]
    data_loader = data_loader.double()
    data_loader -= min_val
    data_loader /= max_val
    return data_loader, min_val, max_val

In [162]:
sample_x

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
        [ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
        [ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
        [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
        [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
        [ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
        [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
        [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
        [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
        [13, 14, 15, 16, 17, 18, 19, 2

In [163]:
normalizeBatch(sample_x)

(tensor([[0.0000, 0.0667, 0.1333, 0.2000, 0.2667, 0.3333, 0.4000, 0.4667, 0.5333,
          0.6000, 0.6667, 0.7333, 0.8000, 0.8667, 0.9333, 1.0000],
         [0.0000, 0.0625, 0.1250, 0.1875, 0.2500, 0.3125, 0.3750, 0.4375, 0.5000,
          0.5625, 0.6250, 0.6875, 0.7500, 0.8125, 0.8750, 0.9375],
         [0.0000, 0.0588, 0.1176, 0.1765, 0.2353, 0.2941, 0.3529, 0.4118, 0.4706,
          0.5294, 0.5882, 0.6471, 0.7059, 0.7647, 0.8235, 0.8824],
         [0.0000, 0.0556, 0.1111, 0.1667, 0.2222, 0.2778, 0.3333, 0.3889, 0.4444,
          0.5000, 0.5556, 0.6111, 0.6667, 0.7222, 0.7778, 0.8333],
         [0.0000, 0.0526, 0.1053, 0.1579, 0.2105, 0.2632, 0.3158, 0.3684, 0.4211,
          0.4737, 0.5263, 0.5789, 0.6316, 0.6842, 0.7368, 0.7895],
         [0.0000, 0.0500, 0.1000, 0.1500, 0.2000, 0.2500, 0.3000, 0.3500, 0.4000,
          0.4500, 0.5000, 0.5500, 0.6000, 0.6500, 0.7000, 0.7500],
         [0.0000, 0.0476, 0.0952, 0.1429, 0.1905, 0.2381, 0.2857, 0.3333, 0.3810,
          0.4286, 0.4762

# Build Generative Adversarial Network

## Discriminator

In [164]:
import torch.nn as nn
import torch.nn.functional as F
import problem_unittests as tests
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [165]:
# helper conv function
def conv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=True):
    """Creates a convolutional layer, with optional batch normalization.
    """
    layers = []
    conv_layer = nn.Conv2d(in_channels, out_channels, 
                           kernel_size, stride, padding, bias=False)
    
    # append conv layer
    layers.append(conv_layer)

    if batch_norm:
        # append batchnorm layer
        layers.append(nn.BatchNorm2d(out_channels))
     
    # using Sequential container
    return nn.Sequential(*layers)

In [166]:
class Discriminator(nn.Module):
    def __init__(self, conv_dim):
        """
        Initialize the Discriminator Module
        :param conv_dim: The depth of the first convolutional layer
        """
        super(Discriminator, self).__init__()

        # complete init function
        self.conv_dim = conv_dim

        # 32x32 input
        self.conv1 = conv(1, conv_dim, 4, batch_norm=False) # first layer, no batch_norm
        # 16x16 out
        self.conv2 = conv(conv_dim, conv_dim*2, 4)
        # 8x8 out
        self.conv3 = conv(conv_dim*2, conv_dim*4, 4)
        # 4x4 out
        
        # final, fully-connected layer
        self.fc = nn.Linear(conv_dim*4*4*4, 1)


    def forward(self, x):
        """
        Forward propagation of the neural network
        :param x: The input to the neural network     
        :return: Discriminator logits; the output of the neural network
        """
        # define feedforward behavior
        # all hidden layers + leaky relu activation
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.conv2(x), 0.2)
        x = F.leaky_relu(self.conv3(x), 0.2)
        
        # flatten
        x = x.view(-1, self.conv_dim*4*4*4)
        
        # final output layer
        x = self.fc(x)            
        return x


# """
# DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
# """
# tests.test_discriminator(Discriminator)

## Generator

The input to the generator is a 15 data points of real data into the LSTM model which outputs the following data point. 

In [177]:
class Generator(nn.Module):
    
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param stock_data_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param stock_prediction_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(Generator, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.dropout = dropout
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.output_size = output_size
        self.n_layers = 1

        # LSTM
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers=1, dropout=dropout, batch_first=True)
        
        #dropout layer before FC layer
#         self.dropout = nn.Dropout(dropout)
        
        #fully-connected output layers
        self.fc = nn.Linear(hidden_layer_size, output_size)
        
#         self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
#                             torch.zeros(1,1,self.hidden_layer_size))
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """       
        
        #LSTM
        r_output, hidden = self.lstm(input_seq.view(len(input_seq) ,1, -1), hidden)
        out_dropout = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
#         out_before_fc = r_output.contiguous().view(-1, self.hidden_dim)
        
        ## TODO: put x through the fully-connected layer
        out_before_fc = r_output.contiguous().view(-1, self.hidden_dim)        

        # reshape into (batch_size, seq_length, output_size)
        batch_size = nn_input.size(0)
        output = out_after_fc.view(batch_size, -1, self.hidden_layer_size)
        # get last batch
        out = output[:, -1]
        return out
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''        
        # initialize hidden state with zero weights, and move to GPU if available
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_layer_size).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_layer_size).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_layer_size).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_layer_size).zero_())
        
        return hidden

# """
# DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
# """
# tests.test_rnn(RNN, train_on_gpu)

In [178]:
def weights_init_normal(m):
    """
    Applies initial weights to certain layers in a model .
    The weights are taken from a normal distribution 
    with mean = 0, std dev = 0.02.
    :param m: A module or layer in a network    
    """
    # classname will be something like:
    # `Conv`, `BatchNorm2d`, `Linear`, etc.
    classname = m.__class__.__name__
    
    # TODO: Apply initial weights to convolutional and linear layers
    
    if classname in ['Conv', 'Linear']:
        nn.init.normal_(m.weight.data, mean = 0, std = 0.02)

In [179]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
def build_network(d_conv_dim, g_conv_dim, z_size):
    # define discriminator and generator
    D = Discriminator(d_conv_dim)
#     stock_data_size, stock_prediction_size, embedding_dim, hidden_dim, n_layers, dropout=0.5
    G = Generator(16, 100, 1, 0.5)

    # initialize model weights
    D.apply(weights_init_normal)
    G.apply(weights_init_normal)

    print(D)
    print()
    print(G)
    
    return D, G


In [180]:
# Define model hyperparams
d_conv_dim = 16
g_conv_dim = 16
z_size = 256

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
D, G = build_network(d_conv_dim, g_conv_dim, z_size)

Discriminator(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc): Linear(in_features=1024, out_features=1, bias=True)
)

Generator(
  (lstm): LSTM(16, 100, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)


##Training on GPU

In [182]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Training on GPU!')

No GPU found. Please use a GPU to train your neural network.


# Discriminator and Generator Losses

d_loss = d_real_loss + d_fake_loss.

In [183]:
def real_loss(D_out):
    '''Calculates how close discriminator outputs are to being real.
       param, D_out: discriminator logits
       return: real loss'''
    batch_size = D_out.size(0)
    labels = torch.ones(batch_size) # real labels = 1
    # move labels to GPU if available     
    if train_on_gpu:
        labels = labels.cuda()
    # binary cross entropy with logits loss
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss

def fake_loss(D_out):
    '''Calculates how close discriminator outputs are to being fake.
       param, D_out: discriminator logits
       return: fake loss'''
    batch_size = D_out.size(0)
    labels = torch.zeros(batch_size) # fake labels = 0
    if train_on_gpu:
        labels = labels.cuda()
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss

# Optimizers

In [184]:
import torch.optim as optim

# Create optimizers for the discriminator D and generator G

# params
lr = 0.0002
beta1=0.5
beta2=0.999 # default value

# Create optimizers for the discriminator and generator
d_optimizer = optim.Adam(D.parameters(), lr, [beta1, beta2])
g_optimizer = optim.Adam(G.parameters(), lr, [beta1, beta2])

# Training 

In [185]:
def train(D, G, n_epochs, print_every=50):
    '''Trains adversarial networks for some number of epochs
       param, D: the discriminator network
       param, G: the generator network
       param, n_epochs: number of epochs to train for
       param, print_every: when to print and record the models' losses
       return: D and G losses'''
    
    # move models to GPU
    if train_on_gpu:
        D.cuda()
        G.cuda()

    # keep track of loss and generated, "fake" samples
    samples = []
    losses = []

    # Get some fixed data for sampling. These are images that are held
    # constant throughout training, and allow us to inspect the model's performance
    sample_size=16
    fixed_z = np.random.uniform(-1, 1, size=(sample_size, z_size))
    fixed_z = torch.from_numpy(fixed_z).float()
    # move z to GPU if available
    if train_on_gpu:
        fixed_z = fixed_z.cuda()

    # epoch training loop
    for epoch in range(n_epochs):

        # batch training loop
        for batch_i, (real_images, _) in enumerate(celeba_train_loader):

            batch_size = real_images.size(0)
            real_images = scale(real_images)

            # ===============================================
            #         YOUR CODE HERE: TRAIN THE NETWORKS
            # ===============================================
            
            # 1. Train the discriminator on real and fake images
            d_optimizer.zero_grad()
        
            # 1. Train with real images

            # Compute the discriminator losses on real images 
            if train_on_gpu:
                real_images = real_images.cuda()

            D_real = D(real_images)
            d_real_loss = real_loss(D_real)

            # 2. Train with fake images

            # Generate fake images
            z = np.random.uniform(-1, 1, size=(batch_size, z_size))
            z = torch.from_numpy(z).float()
            # move x to GPU, if available
            if train_on_gpu:
                z = z.cuda()
            fake_images = G(z)

            # Compute the discriminator losses on fake images            
            D_fake = D(fake_images)
            d_fake_loss = fake_loss(D_fake)

            # add up loss and perform backprop
            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            d_optimizer.step()


            # 2. Train the generator with an adversarial loss
            g_optimizer.zero_grad()
        
            # 1. Train with fake images and flipped labels

            # Generate fake images
            z = np.random.uniform(-1, 1, size=(batch_size, z_size))
            z = torch.from_numpy(z).float()
            if train_on_gpu:
                z = z.cuda()
            fake_images = G(z)

            # Compute the discriminator losses on fake images 
            # using flipped labels!
            D_fake = D(fake_images)
            g_loss = real_loss(D_fake) # use real loss to flip labels

            # perform backprop
            g_loss.backward()
            g_optimizer.step()

            # ===============================================
            #              END OF YOUR CODE
            # ===============================================

            # Print some loss stats
            if batch_i % print_every == 0:
                # append discriminator loss and generator loss
                losses.append((d_loss.item(), g_loss.item()))
                # print discriminator and generator loss
                print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                        epoch+1, n_epochs, d_loss.item(), g_loss.item()))


        ## AFTER EACH EPOCH##    
        # this code assumes your generator is named G, feel free to change the name
        # generate and save sample, fake images
        G.eval() # for generating samples
        samples_z = G(fixed_z)
        samples.append(samples_z)
        G.train() # back to training mode

    # Save training generator samples
    with open('train_samples.pkl', 'wb') as f:
        pkl.dump(samples, f)
    
    # finally return losses
    return losses